<a href="https://colab.research.google.com/github/otufor/mytools/blob/main/TCS_%E5%B9%B4%E6%9C%AB%E8%AA%BF%E6%95%B4%E5%B9%B4%E5%8F%8E%E8%A8%88%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使い方
個人の google drive の pdf(パスワード付き)をアップロードしてください。  
* デフォルトパス : salary/*.pdf

pdfのパスワードをテキストに入力してください。
* デフォルトパス : salary/password.txt

# 機能
* TCSの給与明細・賞与明細の支給額抜き出し・年収予測を行う。
* 給与のpdfが抜けている分は給与平均額で埋める。
* 賞与のpdfが抜けている分は賞与max値で埋める。

## 準備

In [ ]:
# googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# pdf操作ライブラリをインストール
!pip install pdfplumber

## 解析

In [ ]:
pdf_folder_path = "/content/drive/MyDrive/salary/"
password_file_path = "/content/drive/MyDrive/salary/password.txt"

# パスワードを読み込む
with open(password_file_path, 'r') as f:
    password = f.read().strip()

# 正規表現パターン（ファイル名によって選択）
regex_pattern_bonus = r".*支給額"  # 賞与に関する正規表現
regex_pattern_salary = r".*支給額"  # 給与に関する正規表現


In [ ]:
import pdfplumber
import re
import os
import pandas as pd
from IPython.core.display import display, Markdown

def extract_table_from_pdf(pdf_path, password, regex_pattern):
    with pdfplumber.open(pdf_path, password=password) as pdf:
        extracted_data = []
        for page in pdf.pages:
            tables = page.extract_tables()
            for table_index, table in enumerate(tables):
                for row_index, row in enumerate(table):
                    for col_index, cell in enumerate(row):
                        if cell and re.search(regex_pattern, cell):
                            next_cell = table[row_index][col_index + 1] if col_index + 1 < len(row) else None
                            extracted_data.append((page.page_number, cell, next_cell))
    return extracted_data

# 空のDataFrameを作成
df = pd.DataFrame()

# フォルダ内のPDFファイルを順番に処理
for pdf_file in sorted(os.listdir(pdf_folder_path)):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, pdf_file)

        if "賞与" in pdf_file:
            extracted_data = extract_table_from_pdf(pdf_path, password, regex_pattern_bonus)
        elif "給与" in pdf_file:
            extracted_data = extract_table_from_pdf(pdf_path, password, regex_pattern_salary)
        else:
            print(f"Skipping {pdf_file} as it doesn't meet any conditions.")
            continue

        tmpdict = {'Type':'', '年月':'', '総支給額':'', '差引支給額':''}
        if "賞与" in pdf_file:
            tmpdict['Type'] = "賞与"
        if "給与" in pdf_file:
            tmpdict['Type'] = "給与"
        tmpdict['年月'] = f"{pdf_file[-12:-6]}"

        for page_number, cell_data, next_cell_data in extracted_data:
            if next_cell_data is None:
                continue
            if cell_data == '賞総支給額': cell_data='総支給額'
            tmpdict[cell_data] = next_cell_data

        # DataFrameに追加
        df_to_append = pd.DataFrame([tmpdict])
        df = pd.concat([df, df_to_append], ignore_index=True)
# フォーマット変換
df['総支給額']=df['総支給額'].str.replace(',','').astype(int)
df['差引支給額']=df['差引支給額'].str.replace(',','').astype(int)
df['予実']='実績'
# display(df)

# 予測
import math
year=df['年月'][0][:4]
monthly_salary_total=math.floor(df[df['Type']=='給与'].総支給額.mean())
monthly_salary_net_income=math.floor(df[df['Type']=='給与'].差引支給額.mean())
bonus_total=df[df['Type']=='賞与'].総支給額.max()
bonus_net_income=df[df['Type']=='賞与'].差引支給額.max()

# pdfがない月を抽出
## salary
for mm in ["{:02}".format(i) for i in range(1, 13)]:
  if mm not in df[df['Type']=='給与'].年月.str[-2:].to_list():
    df = pd.concat([df, pd.DataFrame([{'Type': '給与', '年月': f'{year}{mm}', '総支給額': monthly_salary_total, '差引支給額': monthly_salary_net_income}])], ignore_index=True)
## bonus
for mm in ["{:02}".format(i) for i in ['12']]:
  if mm not in df[df['Type']=='賞与'].年月.str[-2:].to_list():
    df = pd.concat([df, pd.DataFrame([{'Type': '賞与', '年月': f'{year}{mm}', '総支給額': bonus_total, '差引支給額': bonus_net_income}])], ignore_index=True)
df.loc[df['予実'].isna(), '予実'] = '予測'

# 解析結果表示
def highlight_forecast_row(row):
    if row['予実'] == '予測':
        return ['background-color: gray']*len(row)
    return ['']*len(row)

display(Markdown('# 結果'))
df=df.sort_values(['Type','年月'])
display(df.style.format({'総支給額': '{:,d}', '差引支給額': '{:,d}'}).apply(highlight_forecast_row, axis=1).hide(axis="index"))
# display(df, df[['総支給額','差引支給額']].sum())

# 集計
# 各列の総合計を計算
actual_salary = df.loc[df['予実'] == '実績', '総支給額'].sum()
actual_net_income = df.loc[df['予実'] == '実績', '差引支給額'].sum()
total_salary = df['総支給額'].sum()
total_net_income = df['差引支給額'].sum()

# 総合計値をMarkdownで表示
display(Markdown(f"# 集計"))
display(Markdown(f"- 総支給額の総合計(実績):   \ {actual_salary:,}"))
display(Markdown(f"- 差引支給額の総合計(実績): \ {actual_net_income:,}"))
display(Markdown(f"- 今年の総支給額の総合計(予測):   \ {total_salary:,}"))
display(Markdown(f"- 今年の差引支給額の総合計(予測): \ {total_net_income:,}"))
